# Synthesize Responses from the Retrieved Contexts

## Step 0: Environment Setup

In [1]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph
from libs import create_vector_index, get_entities,embed_entity, vector_search,enhanced_chunk_finder
import pandas as pd
from conn import connect2Googlesheet
from openai import OpenAI
from synthlibs import graph_retriever, parse_relationships, final_context_builder,synthesize_response

# plot figures in high resolution
%config InlineBackend.figure_format = 'retina'
# Force reload of the .env file
try:
    load_dotenv()
    print("The .env file has been loaded")
except Exception as e:
    print("The .env file has not been loaded. Reported error: ", e)

The .env file has been loaded


In [2]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD")
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")
    
# Check if the entities index exists
index_name = "entities"
query = "SHOW INDEXES YIELD name, type WHERE type = 'VECTOR' AND name = $index_name"

result = graph.query(query, params={"index_name": index_name})
if result:
    print("The 'entities' index already exists.")
else:
    create_vector_index(graph, "entities")

Connected to Neo4j database successfully.
The 'entities' index already exists.


## Step 1: Connect to Google Sheet

In [3]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
Q_worksheet = spreadsheet.get_worksheet(2)  
# Select the worksheet: revised summary
Sum_worksheet = spreadsheet.get_worksheet(6) 

# Get all records as a list of dictionaries
ques_data = Q_worksheet.get_all_records()
sum_data = Sum_worksheet.get_all_records()

# Convert to Pandas DataFrame
df_MedQ = pd.DataFrame(ques_data)
df_MedSum = pd.DataFrame(sum_data)

# Display the first few rows of both DataFrames
display(df_MedQ.head())
display(df_MedSum.head())

,condition,number,docs,Question,Mahmud's Note,status,comments,
0,ARDS,1,ACURASYS,Does early administration of neuromuscular blo...,Like,,,
1,ARDS,2,ACURASYS,Do patients with severe ARDS being treated wit...,Replace,fixed,,
2,ARDS,3,ROSE,"In patients with moderate to severe ARDS, does...",Maybe this question: In patients with moderate...,fixed,,
3,ARDS,4,ROSE,Do patients with moderate-to-severe ARDS have ...,Local question (not sure if this is the aim of...,fixed,Wrong concept since PEEP by itself is mandator...,Does the use of neuromuscular blockers in pati...
4,ARDS,5,FACTT,"Among patients with ALI/ARDS, does a conservat...",Local question (not sure if this is the aim of...,fixed,Check if studies defined conservative by CVP <...,


,Doc No,Confirmed by,Topic,Name,Study Design and Methodology,Study Population,Interventions,Comparator,Outcomes,Strengths and Weaknesses,Key Findings and Conclusion
0,1,,ARDS,ACURASYS,"- Multicenter, randomized, double-blind, place...",The study enrolled 340 patients from 20 ICUs i...,- Cisatracurium Group:\n - 3-ml rapid IV infu...,- Placebo group receiving identical IV infusio...,Primary Outcome:\n- 90-day in-hospital mortali...,Strengths:\n- Multicenter design\n- Blinded ra...,Key Findings:\n- Adjusted 90-day survival impr...
1,6,,ARDS,ROSE,"- Multicenter, unblinded, randomized clinical ...",- Patients with moderate-to-severe ARDS\n- Cri...,- 48-hour continuous infusion of cisatracurium...,- Usual care approach\n- No routine neuromuscu...,Primary Outcome:\n- In-hospital death from any...,Strengths:\n- Multicenter design\n- High proto...,Key Findings:\n No significant difference in 9...
2,3,,ARDS,FACTT,"- Prospective, randomized clinical trial\n- Co...","- 1,001 patients with acute lung injury (503 i...",Conservative Strategy:\n- Lower fluid intake\n...,The two fluid management strategies were compa...,Primary Outcome:\n- Death at 60 days\n\nSecond...,Strengths:\n- Randomized design\n- Multi-cente...,Key Findings:\n- No significant difference in ...
3,2,,ARDS,ARDSNet,"- Multicenter, randomized controlled trial\n- ...",- Total Participants: 861 patients\n- Inclusio...,- Lower Tidal Volume Group:\n - Initial tidal...,- Traditional Tidal Volume Group:\n - Initial...,Primary Outcomes:\n- Mortality before hospital...,Strengths:\n- Multicenter design\n- Robust ran...,Key Findings:\n- Mortality reduced by 22% in l...
4,5,,ARDS,PROSEVA,"- Multicenter, prospective, randomized, contro...",- 466 patients with severe ARDS\n- Inclusion c...,- Prone group (237 patients):\n - Turned to p...,- Supine group (229 patients):\n - Remained i...,Primary Outcome:\n- 28-day mortality\n\nSecond...,Strengths:\n- Multicenter design\n- Standardiz...,Key Findings:\n- 28-day mortality: 16.0% (pron...


## Step 2: Build Context before Synthesis

In [4]:
test_question = df_MedQ['Question'][0]
context = graph_retriever(graph, test_question, max_depth=2)
formatted_relations = parse_relationships(context)
print(formatted_relations)

Query: Does early administration of neuromuscular blocking agents increases the ventilator free days?
Found 26 unique relationships:
- furosemide -> prescribed -> administration
- administration -- prescribed -- Lite protocol
- administration -- prescribed -- FACTT Lite
- administration -- prescribed -- FACTT Conservative
- administration -- prescribed -- FACTT Liberal
- administration -- prescribed -- FACTT protocols
- neuromuscular blocking agents -> prescribed -> severely hypoxemic patients
- neuromuscular blocking agents -- prescribed -- prone positioning
- neuromuscular blocking agents -> prescribed -> patients with early, severe ARDS
- neuromuscular blocking agents -- registered -- intensive care unit (ICU)
- neuromuscular blocking agents -- received -- therapy with neuromuscular blocking agents
- neuromuscular blocking agents -> falls_under -> pharmacologic intervention
- neuromuscular blocking agents -> resulted_in -> improve the prognosis of moderate-to-severe ARDS
- neuromusc

In [10]:
final_context , filenames = final_context_builder(graph, test_question, method="hybrid")
#res = get_firstk_chunks(graph, filenames, firstk=3)
print(final_context)

Query: Does early administration of neuromuscular blocking agents increases the ventilator free days?
Found 26 unique relationships:
- furosemide -> prescribed -> administration
- administration -- prescribed -- Lite protocol
- administration -- prescribed -- FACTT Lite
- administration -- prescribed -- FACTT Conservative
- administration -- prescribed -- FACTT Liberal
- administration -- prescribed -- FACTT protocols
- neuromuscular blocking agents -> prescribed -> severely hypoxemic patients
- neuromuscular blocking agents -- prescribed -- prone positioning
- neuromuscular blocking agents -> prescribed -> patients with early, severe ARDS
- neuromuscular blocking agents -- registered -- intensive care unit (ICU)
- neuromuscular blocking agents -- received -- therapy with neuromuscular blocking agents
- neuromuscular blocking agents -> falls_under -> pharmacologic intervention
- neuromuscular blocking agents -> resulted_in -> improve the prognosis of moderate-to-severe ARDS
- neuromusc

## Step 3: Synthesize

### 3.1 Stance Synthesis

In [6]:
response_hy = synthesize_response(graph, test_question, synth_type = 'stance_synthesis', method="hybrid", model="gpt-3.5-turbo")
response_vec= synthesize_response(graph, test_question, synth_type = 'stance_synthesis', method="vector", model="gpt-3.5-turbo")
#response_gr= synthesize_response(graph, test_question, method="graph", model="gpt-3.5-turbo")


In [7]:
print("Hybrid Response: ", response_hy.choices[0].message.content.strip())
print("Vector Response: ", response_vec.choices[0].message.content.strip())
#print("Graph Response: ", response_gr.choices[0].message.content.strip())

Hybrid Response:  {
  "DDS": "Supports",
  "FMWSCPARDS": "Neutral",
  "ESCNBC": "Neutral",
  "APROCCHSS": "Neutral",
  "LSPA": "Neutral",
  "RARDS": "Neutral",
  "ESCPARDS": "Neutral",
  "EDvsLSMENDS": "Neutral",
  "ETSDMV": "Neutral",
  "RDSCI": "Neutral"
}
Vector Response:  {
  "LSPA": "Neutral",
  "ESCPARDS": "Against",
  "DDS": "Supports",
  "ETSDMV": "Neutral",
  "FMWSCPARDS": "Neutral",
  "APROCCHSS": "Neutral",
  "ARDSSRDRFMS": "Neutral",
  "EDvsLSMENDS": "Neutral",
  "RDSCI": "Neutral"
}


### 3.2 PICO Synthesis

In [8]:
response_hy = synthesize_response(graph, test_question, synth_type = 'PICO_synthesis', method="hybrid", model="gpt-3.5-turbo")
response_vec= synthesize_response(graph, test_question, synth_type = 'PICO_synthesis', method="vector", model="gpt-3.5-turbo")

In [9]:
print("Hybrid Response: ", response_hy.choices[0].message.content.strip())
print("Vector Response: ", response_vec.choices[0].message.content.strip())

Hybrid Response:  Based on the provided articles, here are the groupings and summaries:

### Supporting the Claim:
1. **Study 1: Dexmedetomidine and Ventilator-Free Time**
   - **Authors**: Not specified
   - **Year**: Not specified
   - **Title**: Not specified
   - **Journal**: Not specified
   - **Research Question**: Effect of dexmedetomidine on ventilator-free time in critically ill patients.
   - **Type of Study**: Study evaluating the impact of dexmedetomidine on ventilator-free time.
   - **Methodology**: Dexmedetomidine increased ventilator-free time and decreased postoperative complications.
   - **Population Size**: Not specified
   - **Interventions**: Dexmedetomidine administration.
   - **Outcomes**: Increased ventilator-free time and decreased complications.
   - **Key Findings**: Dexmedetomidine increased ventilator-free time and reduced postoperative complications.

### Equivocal or Inconclusive:
1. **Study 2: Comparison of Sedation Agents in ICU Patients**
   - **Auth